# Kepler Mission model training
This section of the notebook trains a neural network based on the contents of the cumulative Kepler exoplanets table.

## Dependencies import

In [ ]:
import pandas as pd
import keras
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
import pathlib
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

## Data preprocessing

In [ ]:
directory = pathlib.Path('datasets_xg')

X_acc, Y_acc = [], []

for file in directory.iterdir():
    df = pd.read_csv(file)
    to_drop_old = ['id', 'goals_period2', 'opposition_goals_period2', 'ballPossession_period2', 'opposition_ballPossession_period2', 'bigChanceCreated_period2', 'opposition_bigChanceCreated_period2', 'totalShotsOnGoal_period2', 'opposition_totalShotsOnGoal_period2', 'goalkeeperSaves_period2', 'opposition_goalkeeperSaves_period2', 'cornerKicks_period2', 'opposition_cornerKicks_period2', 'passes_period2', 'opposition_passes_period2', 'totalTackle_period2', 'opposition_totalTackle_period2', 'freeKicks_period2', 'opposition_freeKicks_period2', 'shotsOnGoal_period2', 'opposition_shotsOnGoal_period2', 'hitWoodwork_period2', 'opposition_hitWoodwork_period2', 'shotsOffGoal_period2', 'opposition_shotsOffGoal_period2', 'blockedScoringAttempt_period2', 'opposition_blockedScoringAttempt_period2', 'totalShotsInsideBox_period2', 'opposition_totalShotsInsideBox_period2', 'totalShotsOutsideBox_period2', 'opposition_totalShotsOutsideBox_period2', 'offsides_period2', 'opposition_offsides_period2', 'accuratePasses_period2', 'opposition_accuratePasses_period2', 'throwIns_period2', 'opposition_throwIns_period2', 'finalThirdEntries_period2', 'opposition_finalThirdEntries_period2', 'accurateLongBalls_period2', 'opposition_accurateLongBalls_period2', 'accurateCross_period2', 'opposition_accurateCross_period2', 'duelWonPercent_period2', 'opposition_duelWonPercent_period2', 'dispossessed_period2', 'opposition_dispossessed_period2', 'groundDuelsPercentage_period2', 'opposition_groundDuelsPercentage_period2', 'aerialDuelsPercentage_period2', 'opposition_aerialDuelsPercentage_period2', 'dribblesPercentage_period2', 'opposition_dribblesPercentage_period2', 'wonTacklePercent_period2', 'opposition_wonTacklePercent_period2', 'interceptionWon_period2', 'opposition_interceptionWon_period2', 'totalClearance_period2', 'opposition_totalClearance_period2', 'goalKicks_period2', 'opposition_goalKicks_period2']
    df = df.drop(columns=['id']).reset_index(drop=True)
    df = df.iloc[::-1].reset_index(drop=True)
    result_counts = df['result'].value_counts()
    min = result_counts.min()

    for value, count in result_counts.items():
        to_trim = count - min
        if to_trim > 0:
            idx = df[df["result"] == value].sample(n=to_trim, random_state=42).index
            df = df.drop(idx)

    Y = df['result']
    X = df.drop(columns=['result']).values.astype(np.float32)

    X_acc.extend(X)
    Y_acc.extend(Y)

X_acc = np.array(X_acc, dtype=np.float32)
Y_acc = np.array(Y_acc)

X_train_unscaled, X_test_unscaled, y_train, y_test = train_test_split(
    X_acc,
    Y_acc,
    test_size=0.2,
    random_state=42,
    stratify=Y_acc
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_unscaled)
X_test_scaled = scaler.transform(X_test_unscaled)

pca = PCA(n_components=0.95)  # keep 95% variance

X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

X_train = X_train_pca
X_test = X_test_pca

joblib.dump(scaler, "kepler_scaler.pkl")
joblib.dump(pca, "kepler_pca.pkl")


In [ ]:
model = RandomForestClassifier(
    n_estimators=500,     # number of trees
    max_depth=None,      # let trees grow fully
    min_samples_leaf=2, # helps generalization
    n_jobs=-1,          # use all CPU cores
    random_state=42
)

model.fit(X_train_unscaled, y_train)

y_pred = model.predict(X_test_unscaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

## Model compilation

In [ ]:
input_layer = layers.Input(shape=(X_train.shape[1],))
norm = layers.BatchNormalization()(input_layer)

x = layers.Dense(128, activation="relu")(input_layer)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

x = layers.Dense(64, activation="relu")(input_layer)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

x = layers.Dense(32, activation="relu")(input_layer)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

x = layers.Dense(16, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

x = layers.Concatenate()([x, norm])

x = layers.Dense(32, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

output = layers.Dense(3, activation="softmax")(x)

model = keras.Model(inputs=input_layer, outputs=output)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

## Training callbacks

In [ ]:
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=50,
    restore_best_weights=True
)

## Model fitting

In [ ]:
model.fit(
    X_train, y_train,
    batch_size=128,
    epochs=1000,
    validation_split=0.1,
    callbacks=[early_stop]
)

In [ ]:
model.save('kepler.keras')

## Testing and model evaluation

In [ ]:
labels = ['HOME', 'AWAY', 'DRAW']

Y_pred_probs = model.predict(X_test)
Y_pred = np.argmax(Y_pred_probs, axis=1)

Y_pred_strings = [labels[i] for i in Y_pred]
Y_true_strings = [labels[i] for i in y_test]

cm = confusion_matrix(Y_true_strings, Y_pred_strings, labels=labels)
print(classification_report(
    y_test,
    Y_pred,
    labels=[0,1,2],
    target_names=labels
))

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()